<a href="https://colab.research.google.com/github/brayan0404/Aprende_Langchain/blob/main/Memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **MEMORY**

Este documento contiene toda la informacion **basica** relevante, a consideracion, acerca de la memoria en Langchain. Estaremos viendo los distintos tipos de memoria que exinten en langchain.

# **Que es la memoria en langchain**

La idea detras de la memoria en langchain es simple, y su funcion es la de todo tipo de memoria, guardar informacion.
Por defecto muchos de los componentes, por miedo a decir todos, son stateless, es decir, sin memoria, esto implica que cuando estas chateando o haciendo completaciones desde un llm estos no tengan un registro de los mensajes anteriores, asi por ejemplo si despues de varios mensajes intercambiados con el llm le preguntas cual fue el primer mensaje que le enviaste, o le haces alguna preunta que tome informacion sobre los mensajes distintos al ultimos el modelo te dirá que no sabe de lo que estas hablando. He aqui donde entra la memoria en juego, lo que viene a resolver es el problema de la perdida de informacion entre interacciones, asi con la memoria el modelo guarda un registro de los mensajes anteriores, y en caso de pregunta sobre un mensaje antiguo este sabra de que le hablas exactamente y podrá brindarte una respuesta basada en el historial de mensajes.

# **ISTALACIONES, CONFIGURACIONES API, IMPORTACIONES**

Para este punto cabe hacer una introducion acerca de los paquetes que estamos importando.

**ConversationChain**

Importamos ConversationChain que es un tipo de chain muy parecido al LLMChain con la diferencia que el primero tiene memoria por defecto, y el segundo no. Al igual que al LLMChain le pasamos un parametro llm y podemos pasar tambien un prompt, y digo podemos, porque por defecto esté trae uno.

**PromptTemplate**

Inportamos PromptTemplate porque vamos a estar creando un prompt perzonaliza (con nuestras propias instruciones y ejemplos si queremos) para sustituir al que por defecto trae el ConversationChain.

### **langchain.memory**

Desde el modulo memory de langchain estaremos importando distintos tipos de memoria que funcionan distinto y cada una tiene sus ventajas y desventajas.

In [ ]:
!pip install langchain
!pip install openai

In [ ]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory, ConversationBufferWindowMemory

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "coloca_tu_clave_de_openai_api_key_aqui"

In [ ]:
llm = OpenAI(
    openai_api_key="OPENAI_API_KEY",
    model_name= "text-davinci-003"
    )


In [ ]:
llm = OpenAI(temperature=0.0)

# **CONVERSATION CHAIN Y LOS DIFERENTES TIPOS DE MEMORIA**

## **EL PROBLEMA DE LA MEMORIA EN LANGCHAIN**

El problema al que me refiero es el de la velocidad y la cantidad de tokens influidos por la memoria, a continuacion explico de que van.

El template del conversation-chain-memoria es el siguiente:

"""
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI: """

Ahora, todos los mensajes se van guardando en la variable history, y que secede?, pues que si los mensajes se guarda en history el tamaño del template va creciendo, y ya sabemos que para cada consulta que hacemos al modelo le pasamos la informacion que tenemos en nuestro template, y como nuestro template ahora es mas grande debido a los mensajes guardados en la history entonces el tiempo de procesamiento será mayor, y como hay mas caracteres tambien será mayor el numero de tokens gastados. Asi a la hora de elegir el tipo de memoria debemos tenee en cuenta al rededos de su funcionamiento estas dos variables.

## **ConversationBufferMemory()**

Este es un tipo de memoria en langchain, y es quizas el mas simple de todos. Cuando el usuario está conversando con el llm este envia al llm texto como input y recibe texto como output, la funcion de este tipo de memoria es guardar todos los inputs y outputs que se han realizado en la conversation en una variable.

### **PROS**
Guardar toda la coneversation le da la capacidad al llm de que cualquier interacion pasada pueda ser referenciada en el presente. Asi si en el mensaje uno le preguntaste sobre que son los llms y en el mensaje 10 le preguntas que si lo que dijo en el mensaje uno es relevante entonces sabra a que mensaje te estas refiriendo y te podra responder.

### **CONS**
Entre mas larga sea la conversation que se tenga que guardar en la memoria el modelo tardará mas en dar una respuesta, y tambien, sera consumidos mas tokens, lo que se traduce en mas dinero gastado, ademas, los llms tienen un limite de tokens, que en openai son de 4096, quiere decir que a partir de aqui el modelo sera insuficiente para guardar mas informacion.


**RECOMENDACION**

Se recomienda usar este tipo de memoria cuando el usuario va a tener no muchas interaciones con el modelo, por ejemplo 10 interaciones, ya que si la conversacion es sostenida en el tiempo incurre en los problemas de la memoria expresados arriba.



In [ ]:
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory()
)

## **ConversationSummaryMemoryMemory()**

Este tipo de memoria hace un resumen sobre todas las interaciones que hemos teneido con el modelo, y cada nueva interacion es resumida e incluida en el resumen. Recibe como parametro el llm ya que este se encarga de hacer el resumen toda la history.

### **PROS**
GRacias a que el modelo resume la history, esto nos permite que la cantidad de informacion guardada sea menor que en la ConversationBufferMemory, y esto trae como beneficios que el modelo responda mas rapido, y que podamos ingresar mas interaciones que en el primero.

### **CONS**

No se recomienda usar este tipo de memoria cuando se van a tener conversaiones cortas, en tanto el numero de tokes es alto para conversaiones cortas en comparacion a otros tipos de memoria, ademas, la capacidad para capturar informacion está directamente ligada a la capacidad de este tipo de memoria de hacer resumenes, es decir, quizas se omita informacion relevante.


**RECOMENDACION**

Se recomienda usar este tipo de memoria para cuando se esperen tener conversasiones largas, ya que aunque al principo el numero de token usados es bastante alto, con el tiempo tiende a parar en crecimiento.

In [ ]:
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationSummaryMemory(llm=llm)
)

## **ConversationBufferWindowMemory(k=n)**

Este tipo de memoria hace una lista de las ultimas k interaciones, donde k es un numero que corresponde al numero de interaciones que queremos guardar, asi si colocamos a k=2 quiere decir que se guardaran solo las dos ultimas dos interaciones que tuvimos con el modelo.

### **PROS**
Las ventajas son claras, al tener limitado el numero de interaciones que podrá recordar estaremos ahorrando tokens y ademas conoceremos el rendimiento del modelo.

### **CONS**
Puede suceder que las interaciones no son suficientes, por lo que podria llegar a ser frustrante, aunque basta con modificar k para solucionarlo.


**RECOMENDACION**
Se recomienda ir cambiando el numero de interaciones segun necesidad.


In [ ]:
conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferWindowMemory(k=2)
)

In [ ]:
conversation("Como me llamo?")

This line of code shows the conversation history

In [ ]:
print(conversation.memory.buffer)

Estos son algunos de los tipos de memoria en langchain, hay otros que quedan por cubrir pero en general son similares.